In [26]:
# Core imports
import numpy as np
from numpy import linalg as la 
import heyoka as hk
import pykep as pk
import pandas as pd
import math
from numpy.lib.function_base import append
from scipy.integrate import odeint

# Sys imports
import time

# Plot imports
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Other imports
import dromo_const as const

%matplotlib notebook

In [27]:
GMe = const.GMe
Re = const.Re
J2 = const.J2

In [28]:
# Auxiliary function
def state2orb(r0, v0, Gparam, angle_unit):
    """
    Converts from state vector to orbital parameters
    units:
    r0 in km
    v0 in km/s
    Gparam in km^3/s^2
    a in km
    e is [-]
    angles in radiants (default) or degrees
    """
    h = np.cross(r0, v0)
    h_norm = np.linalg.norm(h)
    cos_inclination = h[2]/h_norm       # since h scalar product z = h_norm*1*cos(i) = h_3

    if np.linalg.norm(cos_inclination) >= 1:
        cos_inclination = np.sign(cos_inclination)
    inclination = math.acos(cos_inclination)
    inclination_d = math.degrees(inclination)

    if inclination == 0 or inclination == np.pi :
        node_line = [1, 0, 0] # None  # pick the x-axis as your line of Nodes, which is undefined as the orbital and equatorial plane coincide
        RAAN = 0  # None 
        RAAN_r = 0
    else :
        node_line = np.cross([0, 0, 1], h)/(np.linalg.norm(np.cross([0, 0, 1], h))) # cross vector is not commutative
        cos_RAAN = node_line[0]
        if np.linalg.norm(cos_RAAN) >= 1:
            cos_RAAN = np.sign(cos_RAAN)
        RAAN = math.acos(cos_RAAN)
        RAAN_d = math.degrees(RAAN)

    if node_line[1] < 0:
        RAAN = 2*np.pi - RAAN
        RAAN_d = math.degrees(RAAN)

    # From the Laplace vector equation 
    e = (np.cross(v0, h))/Gparam - r0/np.linalg.norm(r0)
    e_norm = np.linalg.norm(e)

    if e_norm < math.pow(10, -5):
        # for circular orbits choose r0 as the apse line to count the true anomaly and define the argument of perigee
        cos_arg_perigee = np.dot(r0, node_line)/np.linalg.norm(r0)
        if np.linalg.norm(cos_arg_perigee) >= 1:
            cos_arg_perigee = np.sign(cos_arg_perigee)
        arg_perigee = math.acos(cos_arg_perigee)
        arg_perigee_d = math.degrees(arg_perigee)
        if r0[2] < 0:
            arg_perigee = 2*np.pi - arg_perigee
            arg_perigee_d = math.degrees(arg_perigee)
        # arg_perigee =  # None 
    else :
        cos_arg_perigee = np.dot(e, node_line)/e_norm
        if np.linalg.norm(cos_arg_perigee) >= 1:
            cos_arg_perigee = np.sign(cos_arg_perigee)
        arg_perigee = math.acos(cos_arg_perigee)
        arg_perigee_d = math.degrees(arg_perigee)
        if e[2] < 0: # e1,e2,e3 dot 0,0,1
            arg_perigee = 2*np.pi - arg_perigee
            arg_perigee_d = math.degrees(arg_perigee)

    perigee = (np.linalg.norm(h)**2/Gparam) * (1/(1+e_norm))
    apogee  = (np.linalg.norm(h)**2/Gparam) * (1/(1-e_norm))

    if apogee < 0:
        # in the case of an hyperbolic orbit
        apogee = - apogee

    semi_major_axis = (perigee+apogee)/2
    T = (2*np.pi/math.sqrt(Gparam)) * math.pow(semi_major_axis, 3/2)  # orbital period (s)

    if e_norm < math.pow(10, -5):
        true_anomaly = 0
    else :
        cos_true_anomaly = np.dot(e, r0)/(e_norm*np.linalg.norm(r0))
        if np.linalg.norm(cos_true_anomaly) >= 1:
            cos_true_anomaly = np.sign(cos_true_anomaly)
        true_anomaly = math.acos(cos_true_anomaly)
        true_anomaly_d = math.degrees(true_anomaly)

    u_r  = r0/np.linalg.norm(r0)
    if np.dot(v0, u_r) < 0:
        # past apogee
        true_anomaly = 2*np.pi - true_anomaly   
        true_anomaly_d = math.degrees(true_anomaly)
        
    if angle_unit == "r":
        return semi_major_axis, e_norm, inclination, RAAN, arg_perigee, true_anomaly
    elif angle_unit == "d":
        return semi_major_axis, e_norm, inclination_d, RAAN_d, arg_perigee_d, true_anomaly_d
    else:
        raise("Insert a valid unit for the angle, r for radians, d for degree")

In [29]:
# create heyoka variables
tau, zeta1, zeta2, zeta3, eta1, eta2, eta3, eta4 = hk.make_vars("tau",
                                                                        "zeta1",
                                                                        "zeta2",
                                                                        "zeta3",
                                                                        "eta1",
                                                                        "eta2",
                                                                        "eta3",
                                                                        "eta4")

# sigma is the independent variable (usually time), so I should not include it as hk variable? 
# (in both cases the error is the same )

In [30]:
# Initial Conditions (dimensional)
# @t=t0*, (x0*, v0*) and (a0*, e0*, i0*, RAAN0*, omega0*, theta0*)
r0d = np.array([0.0, -5888.9727, -3400.0]) #km   (class 'numpy.ndarray')
v0d = np.array([10.691338, 0.0, 0.0])      #km/s
# perigee height should be 6800 km (correct!, as the minimum altitude comes out to be 428 km)
[a0d, e0d, i0d, RAAN0d, omega0d, theta0d] = state2orb(r0d, v0d, GMe, "r")

# from the conversion comes out that a0* = 136,000 km
# and that the orbital and equatorial plane coincide, as i~=0 and RAAN is not defined

# Non-dimensionalise the ICs
r0 = r0d / a0d #[km/km] = [-] since the orbit is highly elliptical normalise with the initial semimajor axis
               # otherwise use r0 = r0d/la.norm(r0d)
v0 = v0d * math.sqrt(a0d/GMe)       #[-]
t0d = 0                                   #s
t0  = t0d / math.sqrt((a0d**3)/GMe) #[-]

# Tranform the non-dimensional ICs (r0, v0) in DROMO elements: (sigma; tau, zeta1, zeta2, zeta3, eta1, eta2, eta3, eta4)
h0 = np.cross(r0, v0)                    # 3-components vector
e0 = - r0/la.norm(r0) - np.cross(h0, v0) # 3-components vector
sigma_0 = 0
tau_0   = t0
zeta1_0 = la.norm(e0)
zeta2_0 = 0
zeta3_0 = 1/la.norm(h0)
eta1_0  = math.sin(i0d/2)*math.cos((RAAN0d-omega0d)/2)
eta2_0  = math.sin(i0d/2)*math.sin((RAAN0d-omega0d)/2)
eta3_0  = math.cos(i0d/2)*math.sin((RAAN0d-omega0d)/2)
eta4_0  = math.cos(i0d/2)*math.cos((RAAN0d-omega0d)/2)

S0 = [tau_0, zeta1_0, zeta2_0, zeta3_0, eta1_0, eta2_0, eta3_0, eta4_0]

In [31]:
# Auxiliary relationships
s = 1 + zeta1 * hk.cos(sigma) + zeta2 * hk.sin(sigma)

In [32]:
# Keplerian motion
apx = 0
apy = 0
apz = 0

In [33]:
# Perturbing accelerations
a_px = apx/(zeta3**4 * s**3)
a_py = apy/(zeta3**4 * s**3)
a_pz = apz/(zeta3**4 * s**3)

In [34]:
# EOMs

dtaudsigma    = 1/(zeta3**3 * s**2)
dzeta1dsigma  = s * hk.sin(sigma) * a_px   + (zeta1 + (1+s)*hk.cos(sigma)) * a_py
dzeta2dsigma  = - s * hk.cos(sigma) * a_px + (zeta2 + (1+s)*hk.sin(sigma)) * a_py
dzeta3dsigma  = zeta3 * a_pz
deta1dsigma   = 1/2 * a_pz * (eta4 * hk.cos(sigma) - eta3 * hk.sin(sigma))
deta2dsigma   = 1/2 * a_pz * (eta3 * hk.cos(sigma) + eta4 * hk.sin(sigma))
deta3dsigma   = 1/2 * a_pz * (-eta2* hk.cos(sigma) + eta1 * hk.sin(sigma))
deta4dsigma   = 1/2 * a_pz * (-eta1* hk.cos(sigma) - eta2 * hk.sin(sigma))

In [35]:
# Define time

# tf = 288.12768941*24*3600            # s
tf = 2*np.pi * 50                      # roughly 50 orbits
delta_t = 1
n_steps = math.floor((tf-t0)/delta_t - 1)
# duration of integration in seconds
tspan = np.linspace(0, tf, n_steps)

In [25]:
print('Compiling the Taylor integrator ... (this is done only once)')
start_time = time.time()

ta = hk.taylor_adaptive(sys = [(tau,dtaudsigma),(zeta1,dzeta1dsigma),
                               (zeta2,dzeta2dsigma),(zeta3,dzeta3dsigma),
                               (eta1,deta1dsigma),(eta2,deta2dsigma),
                               (eta3,deta3dsigma),(eta4,deta4dsigma)],
                                state = S0,
                                time = sigma_0,
                                tol = 1e-9,
                                compact_mode = True)
end_time = time.time()

print('Done, in');
print("--- %s seconds ---" % (end_time - start_time))

print("\nHeyoka Taylor integrator:\n", ta)

Compiling the Taylor integrator ... (this is done only once)


ValueError: Error in the Taylor decomposition of a system of equations: the variable 'sigma' appears in the right-hand side but not in the left-hand side

In [ ]:
# Propagate for 5 time units.
status, min_h, max_h, nsteps = ta.propagate_for(delta_t = 5.)

print("Outcome      : {}".format(status))
print("Min. timestep: {}".format(min_h))
print("Max. timestep: {}".format(max_h))
print("Num. of steps: {}".format(nsteps))
print("Current time : {}\n".format(ta.time))